In [1]:
import json
import base64
import os
import tiktoken
import random
from dataclasses import dataclass
from typing import List, Optional, Dict, Any
from dotenv import load_dotenv
from openai import AzureOpenAI
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from PIL import Image
import numpy as np
import azure.cognitiveservices.speech as speechsdk
import requests
import pygame
import time
from pathlib import Path
import threading
import soundfile as sf
import sounddevice as sd

# 환경 변수 로드
load_dotenv()

ModuleNotFoundError: No module named 'pygame'

# Data Class

In [2]:
@dataclass
class StrangeResponse:
    """이상한 답변을 저장하는 데이터 클래스"""
    question: str
    answer: str
    timestamp: str
    severity: str  # "mild", "moderate", "severe"
    emotion: str = "중립"
    answer_quality: str = "normal"

@dataclass
class ConversationTurn:
    """대화 턴을 저장하는 데이터 클래스"""
    question: str
    answer: str
    timestamp: str
    emotion: str = "중립"
    answer_length: int = 0
    answer_quality: str = "normal"
    audio_file: str = ""  # 음성 파일 경로 추가

class Config:
    """시스템 설정"""
    # Azure OpenAI 설정
    ENDPOINT = os.getenv("gpt-endpoint")
    DEPLOYMENT = "gpt-4o"
    SUBSCRIPTION_KEY = os.getenv("gpt-key")
    API_VERSION = "2024-02-15-preview"
    
    # Azure Speech 설정
    SPEECH_KEY = os.getenv("speech-key")
    SPEECH_REGION = "eastus"
    
    # 토큰 제한
    MAX_TOKENS = 4000

# Image Analysis

In [3]:
class ImageAnalyzer:
    """GPT-4o를 사용한 이미지 분석"""
    
    def __init__(self):
        self.client = AzureOpenAI(
            api_version=Config.API_VERSION,
            azure_endpoint=Config.ENDPOINT,
            api_key=Config.SUBSCRIPTION_KEY,
        )
    
    def analyze_image(self, image_path):
        """이미지 분석"""
        try:
            with open(image_path, "rb") as image_file:
                base64_image = base64.b64encode(image_file.read()).decode('utf-8')
        except Exception:
            return None
        
        try:
            response = self.client.chat.completions.create(
                model=Config.DEPLOYMENT,
                messages=[{
                    "role": "user",
                    "content": [{
                        "type": "text",
                        "text": """이미지를 분석해서 JSON으로 답해주세요:
{
    "caption": "전체 설명",
    "dense_captions": ["세부 설명1", "세부 설명2"],
    "mood": "분위기",
    "time_period": "시대",
    "key_objects": ["객체1", "객체2"],
    "people_description": "인물 설명",
    "people_count": 숫자,
    "time_of_day": "시간대"
}"""
                    }, {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
                    }]
                }],
                max_tokens=1000,
                temperature=0.3
            )
            
            response_text = response.choices[0].message.content
            
            # JSON 추출
            if "```json" in response_text:
                json_start = response_text.find("```json") + 7
                json_end = response_text.find("```", json_start)
                response_text = response_text[json_start:json_end].strip()
            elif "{" in response_text:
                json_start = response_text.find("{")
                json_end = response_text.rfind("}") + 1
                response_text = response_text[json_start:json_end]
            
            return json.loads(response_text)
            
        except Exception:
            return None


# Chat System


In [4]:
class ChatSystem:
    """자연스러운 질문 통합 채팅 시스템 - 토큰 효율 개선"""
    
    def __init__(self):
        self.client = AzureOpenAI(
            api_version=Config.API_VERSION,
            azure_endpoint=Config.ENDPOINT,
            api_key=Config.SUBSCRIPTION_KEY,
        )
        
        self.conversation_history = []
        self.tokenizer = tiktoken.get_encoding("cl100k_base")
        self.token_count = 0
        self.MAX_TOKENS = Config.MAX_TOKENS
        self.conversation_turns = []
        self.last_question = ""
        
        # 음성 녹음 관련 설정
        self.recording = False
        self.audio_thread = None
        self.audio_data = []
        self.sample_rate = 44100
        
        # 음성 파일 저장 디렉토리 생성
        self.audio_dir = Path("audio_records")
        self.audio_dir.mkdir(exist_ok=True)
    
    def start_recording(self):
        """음성 녹음 시작"""
        if self.recording:
            return
        
        self.recording = True
        self.audio_data = []
        
        def audio_callback(indata, frames, time, status):
            if status:
                print(f"Status: {status}")
            if self.recording:
                self.audio_data.append(indata.copy())
        
        self.audio_thread = sd.InputStream(
            samplerate=self.sample_rate,
            channels=1,
            callback=audio_callback
        )
        self.audio_thread.start()
    
    def stop_recording(self):
        """음성 녹음 중지 및 파일 저장"""
        if not self.recording:
            return None
        
        self.recording = False
        if self.audio_thread:
            self.audio_thread.stop()
            self.audio_thread.close()
            self.audio_thread = None
        
        if not self.audio_data:
            return None
        
        # 녹음된 데이터를 하나의 배열로 합치기
        audio_data = np.concatenate(self.audio_data, axis=0)
        
        # 파일명 생성 (timestamp 사용)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = self.audio_dir / f"record_{timestamp}.wav"
        
        # WAV 파일로 저장
        sf.write(filename, audio_data, self.sample_rate)
        
        return str(filename)
        
    def setup_conversation_context(self, analysis_result):
        """대화 컨텍스트 설정"""
        caption = analysis_result.get("caption", "")
        dense_captions = analysis_result.get("dense_captions", [])
        mood = analysis_result.get("mood", "")
        time_period = analysis_result.get("time_period", "")
        key_objects = analysis_result.get("key_objects", [])
        people_description = analysis_result.get("people_description", "")
        people_count = analysis_result.get("people_count", 0)
        time_of_day = analysis_result.get("time_of_day", "")
        
        dense_captions_text = "\n".join([f"- {dc}" for dc in dense_captions])
        key_objects_text = ", ".join(key_objects)
        
        system_message = f"""너는 노인과 대화하는 요양보호사야. 노인과 특정 이미지에 대해서 질의응답을 주고받아. 
노인은 치매 증상이 갑자기 나타날 수도 있어. 반복되는 말에도 똑같이 대답해줘야 해. 
친절하고 어른을 공경하는 말투여야 해. 그리고 공감을 잘 해야 해. 예의도 지켜. 
너는 주로 질문을 하는 쪽이고, 노인은 대답을 해줄거야. 대답에 대한 리액션과 함께 적절히 대화를 이어 가.
노인의 발언이 끝나면 그와 관련된 공감 문장을 먼저 말한 후, 자연스럽게 그 기억에 대해 더 물어보는 꼬리 질문을 덧붙여. 하지만 메인 주제는 주어진 이미지 정보에 대해 어르신께 대화 문맥에 맞춰 자연스럽게 질문하는 거야.

=== 이미지 정보 ===
주요 설명: {caption}
분위기/감정: {mood}
추정 시대: {time_period}
시간대: {time_of_day}
인원 수: {people_count}명
주요 객체들: {key_objects_text}
인물 설명: {people_description}

세부 요소들:
{dense_captions_text}

=== 대화 원칙 ===
간결하게: 50자 이내로 질문하기
사진: 대화도 대화지만 사진에 대한 주제에서 벗아나진 말아줘
심도있는: 사람과 깊고 의미있게 대화하기 
흥미롭게: 이미지에 대한 흥미로운 질문을 먼저 던져 대화를 시작하세요
공감하기: 사진에 대하여 공감을 하고 친근하게 대화
하나씩만: 한 번에 질문 하나만
자연스럽게: 답변에 따라 연관 질문
따뜻하게: 공감 후 질문, 사람의 마음을 따듯하게 해주는 대화들"""
        
        self.conversation_history = [{"role": "system", "content": system_message}]
        self.token_count = len(self.tokenizer.encode(system_message))
    
    def generate_initial_question(self):
        """첫 질문 생성"""
        response = self.client.chat.completions.create(
            model=Config.DEPLOYMENT,
            messages=self.conversation_history + [
                {"role": "user", "content": "어르신께 따듯하고 친근하게 사진에 대하여 질문을 해주세요. 50자 이내로 간결하게 질문해주세요."}
            ],
            max_tokens=512,
            temperature=0.8
        )
        
        initial_question = response.choices[0].message.content
        self.conversation_history.append({"role": "assistant", "content": initial_question})
        self.token_count += len(self.tokenizer.encode(initial_question))
        self.last_question = initial_question
        
        return initial_question

    def chat_about_image(self, user_query, with_audio=False):
        """대화 처리"""
        user_tokens = len(self.tokenizer.encode(user_query))
        
        # 음성 녹음 시작 (if requested)
        audio_file = None
        if with_audio:
            self.start_recording()
        
        # 대화 턴 저장
        if self.last_question:
            # 음성 녹음 중지 및 파일 저장 (if recording)
            if with_audio:
                audio_file = self.stop_recording()
            
            conversation_turn = ConversationTurn(
                question=self.last_question,
                answer=user_query,
                timestamp=datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                answer_length=len(user_query.strip()),
                audio_file=audio_file if audio_file else ""
            )
            self.conversation_turns.append(conversation_turn)
        
        self.conversation_history.append({"role": "user", "content": user_query})
        self.token_count += user_tokens
        
        # 토큰 제한 확인
        if self.token_count > self.MAX_TOKENS:
            answer = "대화 시간이 다 되었어요. 수고하셨습니다."
            self.conversation_history.append({"role": "assistant", "content": answer})
            return answer, True
        
        # AI 응답 생성
        response = self.client.chat.completions.create(
            model=Config.DEPLOYMENT,
            messages=self.conversation_history,
            max_tokens=1024,
            temperature=0.7
        )
        answer = response.choices[0].message.content
        
        self.conversation_history.append({"role": "assistant", "content": answer})
        self.token_count += len(self.tokenizer.encode(answer))
        self.last_question = answer
        
        if self.token_count > self.MAX_TOKENS:
            return answer, True
        
        return answer, False

# Voice System Class

In [5]:
class VoiceSystem:
    """음성 입출력 시스템"""
    
    def __init__(self):
        self.speech_key = Config.SPEECH_KEY
        self.region = Config.SPEECH_REGION
        
        # STT 설정
        self.speech_config = speechsdk.SpeechConfig(subscription=self.speech_key, region=self.region)
        self.speech_config.speech_recognition_language = "ko-KR"
        
        # TTS 설정
        self.tts_voice = "ko-KR-SunHiNeural"
        
        # 오디오 폴더
        self.audio_dir = Path("audio_files")
        self.audio_dir.mkdir(exist_ok=True)
        
        # pygame 초기화
        try:
            pygame.mixer.init()
            self.audio_enabled = True
        except:
            self.audio_enabled = False
    
    def transcribe_speech(self) -> str:
        """STT: 음성을 텍스트로 변환"""
        try:
            audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
            speech_recognizer = speechsdk.SpeechRecognizer(
                speech_config=self.speech_config, 
                audio_config=audio_config
            )
            
            print("🎙️ 말씀해 주세요...")
            result = speech_recognizer.recognize_once()
            
            if result.reason == speechsdk.ResultReason.RecognizedSpeech:
                recognized_text = result.text.strip()
                print(f"👤 \"{recognized_text}\"")
                
                # 종료 명령어 감지
                exit_commands = ['종료', '그만', '끝', '나가기', 'exit', 'quit', 'stop']
                cleaned_text = recognized_text.lower().replace(' ', '').replace('.', '')
                
                for exit_cmd in exit_commands:
                    if exit_cmd.lower() in cleaned_text:
                        return "종료"
                
                return recognized_text
            else:
                print("❌ 음성을 인식할 수 없습니다. 다시 말씀해 주세요.")
                return ""
        except Exception:
            return ""
    
    def get_access_token(self):
        """Azure Speech Service 액세스 토큰 요청"""
        url = f"https://{self.region}.api.cognitive.microsoft.com/sts/v1.0/issueToken"
        headers = {"Ocp-Apim-Subscription-Key": self.speech_key}
        try:
            res = requests.post(url, headers=headers)
            res.raise_for_status()
            return res.text
        except Exception:
            return None
    
    def synthesize_speech(self, text: str) -> str:
        """TTS: 텍스트를 음성으로 변환하고 재생"""
        if not text.strip():
            return None
            
        try:
            token = self.get_access_token()
            if not token:
                return None
                
            tts_url = f"https://{self.region}.tts.speech.microsoft.com/cognitiveservices/v1"
            
            headers = {
                "Authorization": f"Bearer {token}",
                "Content-Type": "application/ssml+xml",
                "X-Microsoft-OutputFormat": "riff-16khz-16bit-mono-pcm",
                "User-Agent": "DementiaAnalysisSystem"
            }
            
            ssml = f"""
            <speak version='1.0' xml:lang='ko-KR'>
                <voice xml:lang='ko-KR' xml:gender='Female' name='{self.tts_voice}'>
                    {text}
                </voice>
            </speak>
            """
            
            res = requests.post(tts_url, headers=headers, data=ssml.encode("utf-8"))
            res.raise_for_status()
            
            # 음성 파일 저장
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            output_path = self.audio_dir / f"tts_{timestamp}.wav"
            
            with open(output_path, "wb") as f:
                f.write(res.content)
            
            # 음성 재생
            if self.audio_enabled:
                try:
                    pygame.mixer.music.load(str(output_path))
                    pygame.mixer.music.play()
                    while pygame.mixer.music.get_busy():
                        time.sleep(0.1)
                except Exception:
                    pass
            
            return str(output_path)
            
        except Exception:
            return None


# Story Telling / Report System

In [6]:
class StoryGenerator:
    def __init__(self, chat_system):
        self.chat_system = chat_system
        self.client = chat_system.client
        self.strange_responses = []
        self.rule_based_alerts = []
        self.conversation_id = ""
    
    def _create_conversation_folders(self, image_path):
        image_basename = os.path.splitext(os.path.basename(image_path))[0]
        
        # conversation_log/{이미지명}/ 폴더 생성
        image_dir = Path("conversation_log") / image_basename
        image_dir.mkdir(parents=True, exist_ok=True)
        
        # 기존 대화 폴더들 확인하여 다음 번호 결정
        existing_dirs = list(image_dir.glob(f"{image_basename}_conv*"))
        conv_number = len(existing_dirs) + 1
        
        # 대화 ID: {이미지명}_conv{번호}
        self.conversation_id = f"{image_basename}_conv{conv_number}"
        
        # 대화별 폴더: {이미지명}_conv{번호}/
        conversation_dir = image_dir / self.conversation_id
        conversation_dir.mkdir(exist_ok=True)
        
        print(f"📁 저장 구조:")
        print(f"   메인 폴더: conversation_log/{image_basename}/{self.conversation_id}/")
        print(f"   대화 파일: {self.conversation_id}.txt")
        return conversation_dir
    
    def _save_individual_qa_pairs(self, conversation_dir):
        """개별 질의응답 쌍 저장 - 간소화된 형식"""
        for i, turn in enumerate(self.chat_system.conversation_turns, 1):
            qa_filename = conversation_dir / f"qa_{i:02d}.txt"
            
            with open(qa_filename, 'w', encoding='utf-8') as f:
                f.write(f"=== 질의응답 {i}번 ===\n")
                f.write(f"대화 ID: {self.conversation_id}\n")
                f.write(f"시간: {turn.timestamp}\n")
                f.write(f"{'='*25}\n\n")
                f.write(f"🤖 질문:\n{turn.question}\n\n")
                f.write(f"👤 답변:\n{turn.answer}\n")
                f.write(f"{'='*25}\n")
    
    def _load_qa_pairs_for_report(self, pairs_dir):
        qa_files = sorted([f for f in pairs_dir.glob("qa_*.txt")])
        qa_data = []
        for qa_file in qa_files:
            try:
                with open(qa_file, 'r', encoding='utf-8') as f:
                    qa_data.append({'file': qa_file.name, 'content': f.read()})
            except Exception:
                continue
        return qa_data
    
    def analyze_speech_patterns(self):
        if not self.chat_system.conversation_turns:
            return
        
        patterns = {
            'severe_depression': ["죽고싶", "살기싫", "의미없", "포기하고싶", "지쳤", "힘들어죽겠", "세상이싫", "절망"],
            'severe_anxiety': ["무서워죽겠", "불안해미쳐", "걱정돼죽겠", "두려워", "숨막혀", "공황", "패닉"],
            'severe_anger': ["화나죽겠", "미쳐버리겠", "짜증나죽겠", "열받아", "빡쳐", "분해", "참을수없"],
            'cognitive_decline': ["기억안나", "모르겠", "잊어버렸", "생각안나", "까먹었", "헷갈려", "누구였는지", "몰라"]
        }
        
        memory_issues = very_short_answers = meaningless_answers = 0
        repetitive_patterns = []
        
        for i, turn in enumerate(self.chat_system.conversation_turns):
            answer = turn.answer.replace(" ", "").lower()
            
            for pattern_type, keywords in patterns.items():
                for keyword in keywords:
                    if keyword in answer:
                        severity = "critical" if pattern_type == 'severe_depression' else "high"
                        self.rule_based_alerts.append({
                            "type": pattern_type,
                            "turn_number": i + 1,
                            "keyword": keyword,
                            "answer": turn.answer,
                            "timestamp": turn.timestamp,
                            "severity": severity
                        })
                        if pattern_type == 'cognitive_decline':
                            memory_issues += 1
            
            if len(turn.answer.strip()) <= 5:
                very_short_answers += 1
            
            if turn.answer.strip() in ["음", "어", "그냥", "네", "아니", "응", "어?"]:
                meaningless_answers += 1
            
            if i >= 3:
                recent_answers = [t.answer.strip() for t in self.chat_system.conversation_turns[i-3:i]]
                if turn.answer.strip() in recent_answers:
                    repetitive_patterns.append(i + 1)
        
        total_turns = len(self.chat_system.conversation_turns)
        
        thresholds = [
            (memory_issues >= total_turns * 0.7, "severe_memory_loss", "critical", f"전체 {total_turns}회 중 {memory_issues}회 기억 문제"),
            (very_short_answers >= total_turns * 0.8, "communication_difficulty", "high", f"전체 {total_turns}회 중 {very_short_answers}회 짧은 답변"),
            (meaningless_answers >= total_turns * 0.6, "cognitive_confusion", "high", f"전체 {total_turns}회 중 {meaningless_answers}회 무의미한 답변"),
            (len(repetitive_patterns) >= 3, "repetitive_behavior", "moderate", f"답변 반복 {len(repetitive_patterns)}회")
        ]
        
        for condition, alert_type, severity, description in thresholds:
            if condition:
                self.rule_based_alerts.append({"type": alert_type, "description": description, "severity": severity})

    def calculate_ratings(self):
        total_responses = len(self.chat_system.conversation_turns)
        strange_count = len(self.strange_responses)
        
        if total_responses == 0:
            return {"emotion": 3, "coherence": 3, "overall": 3}
        
        emotions = [turn.emotion for turn in self.chat_system.conversation_turns if hasattr(turn, 'emotion')]
        emotion_counts = {}
        for emotion in emotions:
            emotion_counts[emotion] = emotion_counts.get(emotion, 0) + 1
        
        positive_emotions = ["기쁨", "그리움", "감사", "애정", "흥미"]
        negative_emotions = ["슬픔", "무력감", "우울감", "분노", "불안", "짜증"]
        
        positive_count = sum(emotion_counts.get(e, 0) for e in positive_emotions)
        negative_count = sum(emotion_counts.get(e, 0) for e in negative_emotions)
        
        critical_emotion_alerts = [alert for alert in self.rule_based_alerts 
                                 if alert.get('severity') == 'critical' and 
                                 alert.get('type') in ['severe_depression', 'severe_anxiety', 'severe_anger']]
        
        if len(critical_emotion_alerts) > 0:
            emotion_rating = 1
        elif negative_count > positive_count * 2:
            emotion_rating = 2
        elif negative_count > positive_count:
            emotion_rating = 3
        elif positive_count > negative_count:
            emotion_rating = 4
        else:
            emotion_rating = 5 if positive_count > negative_count * 2 else 3
        
        strange_percentage = (strange_count / total_responses * 100) if total_responses > 0 else 0
        severe_count = sum(1 for resp in self.strange_responses if resp.severity == 'severe')
        
        if strange_percentage == 0:
            coherence_rating = 5
        elif strange_percentage <= 20 and severe_count == 0:
            coherence_rating = 4
        elif strange_percentage <= 40 and severe_count <= 1:
            coherence_rating = 3
        elif strange_percentage <= 60 or severe_count <= 2:
            coherence_rating = 2
        else:
            coherence_rating = 1
        
        answer_qualities = [turn.answer_quality for turn in self.chat_system.conversation_turns if hasattr(turn, 'answer_quality')]
        quality_counts = {"poor": 0, "normal": 0, "good": 0, "excellent": 0}
        for quality in answer_qualities:
            quality_counts[quality] += 1
        
        excellent_percentage = (quality_counts["excellent"] / total_responses * 100) if total_responses > 0 else 0
        good_percentage = (quality_counts["good"] / total_responses * 100) if total_responses > 0 else 0
        poor_percentage = (quality_counts["poor"] / total_responses * 100) if total_responses > 0 else 0
        
        critical_cognitive_alerts = [alert for alert in self.rule_based_alerts 
                                   if alert.get('severity') == 'critical' and 
                                   alert.get('type') in ['severe_memory_loss', 'communication_difficulty']]
        
        if len(critical_cognitive_alerts) > 0 or poor_percentage >= 50:
            overall_rating = 1
        elif poor_percentage >= 30 or (strange_percentage > 50 and severe_count >= 2):
            overall_rating = 2
        elif excellent_percentage >= 30 or (good_percentage >= 50 and strange_percentage <= 20):
            overall_rating = 5
        elif good_percentage >= 30 or strange_percentage <= 30:
            overall_rating = 4
        else:
            overall_rating = 3
        
        return {"emotion": emotion_rating, "coherence": coherence_rating, "overall": overall_rating}
    
    def format_star_rating(self, rating):
        stars = "⭐" * rating + "☆" * (5 - rating)
        return f"{stars} ({rating}/5)"

    def analyze_entire_conversation(self):
        if not self.chat_system.conversation_turns:
            return
        
        self.strange_responses = []
        self.rule_based_alerts = []
        self.analyze_speech_patterns()
        
        conversation_text = ""
        for i, turn in enumerate(self.chat_system.conversation_turns, 1):
            conversation_text += f"[{i}] 질문: {turn.question}\n답변: {turn.answer} (길이: {turn.answer_length}자)\n\n"
        
        analysis_prompt = f"""치매 환자 대화 분석하여 JSON 응답:
{conversation_text}

JSON: {{"conversation_analysis": [{{"turn_number": 1, "is_strange": true/false, "severity": "normal/mild/moderate/severe", "emotion": "감정", "answer_quality": "poor/normal/good/excellent", "reason": "이유"}}], "overall_assessment": {{"dominant_emotion": "주요감정", "cognitive_level": "normal/mild_concern/moderate_concern/severe_concern"}}}}

감정: 기쁨,슬픔,그리움,무력감,우울감,분노,불안,중립,감사,애정,흥미,짜증"""

        try:
            response = self.client.chat.completions.create(
                model=Config.DEPLOYMENT,
                messages=[
                    {"role": "system", "content": "치매 환자 대화 분석 전문 AI"},
                    {"role": "user", "content": analysis_prompt}
                ],
                max_tokens=1024,
                temperature=0.1
            )
            
            analysis_text = response.choices[0].message.content
            
            if "```json" in analysis_text:
                json_start = analysis_text.find("```json") + 7
                json_end = analysis_text.find("```", json_start)
                analysis_text = analysis_text[json_start:json_end].strip()
            elif "{" in analysis_text:
                json_start = analysis_text.find("{")
                json_end = analysis_text.rfind("}") + 1
                analysis_text = analysis_text[json_start:json_end]
            
            analysis_result = json.loads(analysis_text)
            
            conversation_analyses = analysis_result.get("conversation_analysis", [])
            for i, analysis in enumerate(conversation_analyses):
                if i < len(self.chat_system.conversation_turns):
                    turn = self.chat_system.conversation_turns[i]
                    turn.emotion = analysis.get("emotion", "중립")
                    turn.answer_quality = analysis.get("answer_quality", "normal")
                    
                    if analysis.get("is_strange", False):
                        strange_response = StrangeResponse(
                            question=turn.question,
                            answer=turn.answer,
                            timestamp=turn.timestamp,
                            severity=analysis.get("severity", "mild"),
                            emotion=turn.emotion,
                            answer_quality=turn.answer_quality
                        )
                        self.strange_responses.append(strange_response)
            
            return analysis_result
            
        except Exception as e:
            return None
        
    def generate_story_from_conversation(self, image_path):
        conversation_text = ""
        for turn in self.chat_system.conversation_turns:
            conversation_text += f"질문: {turn.question}\n답변: {turn.answer}\n\n"
        
        if not conversation_text.strip():
            return None, None
        
        story_prompt = f"""대화 기반으로 어르신 1인칭 추억 스토리 15줄 작성:
{conversation_text}
지침: 답변 기반 작성, 감정과 감각 포함, 따뜻한 톤, 손자/손녀에게 들려주는 어투"""
        
        try:
            response = self.client.chat.completions.create(
                model=Config.DEPLOYMENT,
                messages=[
                    {"role": "system", "content": "노인 추억 스토리텔러"},
                    {"role": "user", "content": story_prompt}
                ],
                max_tokens=512,
                temperature=0.8
            )
            
            story = response.choices[0].message.content
            story_dir = "story_telling"
            os.makedirs(story_dir, exist_ok=True)
            
            image_basename = os.path.splitext(os.path.basename(image_path))[0]
            story_filename = os.path.join(story_dir, f"{image_basename}_story.txt")
            
            with open(story_filename, 'w', encoding='utf-8') as f:
                f.write(story)
            
            return story, story_filename
            
        except Exception:
            return None, None
    
    def save_conversation_summary(self, conversation_dir=None):
        if conversation_dir:
            qa_data = self._load_qa_pairs_for_report(conversation_dir)
        
        analysis_result = self.analyze_entire_conversation()
        total_responses = len(self.chat_system.conversation_turns)
        strange_count = len(self.strange_responses)
        
        if total_responses == 0:
            return "대화가 진행되지 않았습니다."
        
        emotions = [turn.emotion for turn in self.chat_system.conversation_turns if hasattr(turn, 'emotion')]
        emotion_counts = {}
        for emotion in emotions:
            emotion_counts[emotion] = emotion_counts.get(emotion, 0) + 1
        
        if emotion_counts:
            dominant_emotion = max(emotion_counts, key=emotion_counts.get)
            positive_emotions = ["기쁨", "그리움", "감사", "애정", "흥미"]
            negative_emotions = ["슬픔", "무력감", "우울감", "분노", "불안", "짜증"]
            
            positive_count = sum(emotion_counts.get(e, 0) for e in positive_emotions)
            negative_count = sum(emotion_counts.get(e, 0) for e in negative_emotions)
            
            if positive_count > negative_count:
                overall_mood = "긍정적"
                mood_icon = "😊"
            elif negative_count > positive_count:
                overall_mood = "부정적" 
                mood_icon = "😔"
            else:
                overall_mood = "중립적"
                mood_icon = "😐"
        else:
            dominant_emotion = "중립"
            overall_mood = "중립적"
            mood_icon = "😐"
        
        critical_alerts = [alert for alert in self.rule_based_alerts if alert.get('severity') == 'critical']
        high_alerts = [alert for alert in self.rule_based_alerts if alert.get('severity') == 'high']
        ratings = self.calculate_ratings()
        
        summary = f"\n{'='*60}\n📋 치매 진단 대화 분석 리포트\n{'='*60}\n"
        summary += f"📅 분석 일시: {datetime.now().strftime('%Y년 %m월 %d일 %H:%M:%S')}\n"
        summary += f"🆔 대화 ID: {self.conversation_id}\n{'='*60}\n\n"
        
        summary += f"🎯 종합 평가\n{'─'*30}\n"
        summary += f"😊 감정 상태:     {self.format_star_rating(ratings['emotion'])}\n"
        summary += f"💬 답변 일관성:   {self.format_star_rating(ratings['coherence'])}\n"
        summary += f"🧠 전반적 인지:   {self.format_star_rating(ratings['overall'])}\n{'─'*30}\n\n"
        
        summary += f"📊 대화 개요\n{'─'*30}\n"
        summary += f"💬 총 대화 횟수: {total_responses}회\n"
        summary += f"{mood_icon} 전반적 감정: {overall_mood} (주요: {dominant_emotion})\n"
        summary += f"{'✅ 어긋난 답변: 없음' if strange_count == 0 else f'⚠️ 어긋난 답변: {strange_count}회'}\n"
        summary += f"{'✅ 발화 패턴: 특이사항 없음' if len(self.rule_based_alerts) == 0 else f'🔍 발화 패턴: {len(self.rule_based_alerts)}건 관찰'}"
        if len(critical_alerts) > 0:
            summary += f" (⚠️ 주의: {len(critical_alerts)}건)"
        summary += f"\n{'─'*30}\n\n"
        
        if strange_count == 0 and len(critical_alerts) == 0:
            summary += f"🎉 대화 결과\n{'─'*30}\n"
            summary += f"✅ 대화 중 특별히 걱정되는 답변은 없었습니다.\n"
            summary += f"💚 어르신께서 안정적으로 잘 응답해주셨어요.\n"
            if len(high_alerts) > 0:
                summary += f"💡 참고: {len(high_alerts)}번의 발화 패턴이 관찰되었습니다.\n"
            summary += f"🌟 지금처럼 따뜻한 환경과 꾸준한 관심 속에 계시면 좋겠습니다.\n"
            summary += f"{'='*60}\n"
            return summary
        
        if len(self.rule_based_alerts) > 0 or strange_count > 0:
            summary += f"🚨 주요 발견사항\n{'─'*30}\n"
            
            if len(self.rule_based_alerts) > 0:
                alert_types = {
                    'severe_depression': '😔 우울한 표현',
                    'severe_anxiety': '😰 불안한 표현', 
                    'severe_anger': '😡 화가 난 표현',
                    'severe_memory_loss': '🧠 기억 관련 어려움',
                    'communication_difficulty': '💬 대화 어려움',
                    'cognitive_confusion': '❓ 혼란스러운 답변',
                    'repetitive_behavior': '🔄 반복되는 답변'
                }
                
                alert_summary = {}
                for alert in self.rule_based_alerts:
                    alert_name = alert_types.get(alert['type'], f"⚠️ {alert['type']}")
                    alert_summary[alert_name] = alert_summary.get(alert_name, 0) + 1
                
                for alert_name, count in alert_summary.items():
                    summary += f"{alert_name}: {count}번\n"
                summary += f"\n"
            
            if strange_count > 0:
                severity_counts = {"mild": 0, "moderate": 0, "severe": 0}
                for response in self.strange_responses:
                    severity_counts[response.severity] += 1
                
                summary += f"🔍 어긋난 답변 분석:\n"
                if severity_counts['mild'] > 0:
                    summary += f"  🟡 조금 어긋남: {severity_counts['mild']}회\n"
                if severity_counts['moderate'] > 0:
                    summary += f"  🟠 꽤 어긋남: {severity_counts['moderate']}회\n"
                if severity_counts['severe'] > 0:
                    summary += f"  🔴 많이 어긋남: {severity_counts['severe']}회\n"
                summary += f"\n"
            
            summary += f"{'─'*30}\n\n"
        
        if strange_count > 0 and strange_count <= 5:
            summary += f"📝 어긋난 답변 상세\n{'─'*30}\n"
            for i, response in enumerate(self.strange_responses, 1):
                summary += f"{i}. {response.timestamp}\n"
                summary += f"   ❓ 질문: {response.question}\n"
                summary += f"   💬 답변: {response.answer}\n"
                summary += f"   😊 상태: {response.emotion} | 🎯 품질: {response.answer_quality}\n\n"
            summary += f"{'─'*30}\n\n"
        
        summary += f"💡 권장사항\n{'─'*30}\n"
        
        if len(critical_alerts) > 0:
            summary += f"🚨 긴급 권장사항:\n   심각한 정신건강 위험 신호가 감지되었습니다.\n   빠른 시일 내로 연락을 드리는 것을 권장합니다.\n\n"
            for alert in critical_alerts:
                if alert['type'] == 'severe_depression':
                    summary += f"   ⚠️ 극심한 우울감 표현 감지\n      → 연락드려 기분전환을 도와드리세요.\n\n"
                elif alert['type'] == 'severe_memory_loss':
                    summary += f"   ⚠️ 심각한 기억력 저하 감지\n      → 가족과 함께 추억을 되새겨보세요.\n\n"
        elif len(high_alerts) >= 2:
            summary += f"⚠️ 주의 권장사항:\n   최근 대화에서 혼란스러운 답변이 자주 보였습니다.\n   가족과 함께 이야기를 나눠보시길 권장합니다.\n\n"
        elif len(high_alerts) >= 1:
            summary += f"🔶 일반 권장사항:\n   약간 걱정되는 답변이 있었습니다.\n   시간을 내어 안부 전화를 드려보세요.\n\n"
        elif strange_count > 0:
            summary += f"💙 관심 권장사항:\n   전반적으로 잘 응답해주셨지만, 간혹 어긋난 답변이 보입니다.\n   가볍게라도 주변의 관심과 확인이 있으면 좋겠습니다.\n\n"
        else:
            summary += f"💚 훌륭한 상태:\n   어르신께서 무척 안정적으로 잘 응답해주셨습니다.\n   지금처럼 따뜻한 환경과 꾸준한 관심을 유지해주세요.\n\n"
        
        summary += f"🏠 가족을 위한 조언\n{'─'*30}\n"
        
        emotion_advice = {
            "짜증": "🔴 최근 짜증스러운 감정을 표현하셨어요.\n   → 감정을 자연스럽게 표현하도록 따뜻하게 공감해주세요.\n   → 요즘 어떠신지 자주 안부를 여쭤보시면 큰 힘이 됩니다.",
            "우울감": "🟠 슬픔이나 우울감을 표현하셨어요.\n   → 함께 옛 추억을 나누거나 좋아하시던 이야기를 꺼내보세요.\n   → 감정을 안정시키는 데 도움이 될 수 있습니다.",
            "슬픔": "🟠 슬픔이나 우울감을 표현하셨어요.\n   → 함께 옛 추억을 나누거나 좋아하시던 이야기를 꺼내보세요.\n   → 감정을 안정시키는 데 도움이 될 수 있습니다.",
            "무력감": "😞 무기력하거나 소외감을 표현하셨어요.\n   → '어르신 덕분이에요'처럼 인정해드리면 자존감 회복에 도움됩니다.\n   → 함께 의미 있는 활동을 하며 힘이 되어 주세요.",
            "분노": "😡 갑작스럽게 화를 내시거나 강한 어조를 보이셨어요.\n   → 감정 뒤에 불안이나 혼란감이 있을 수 있으니 조용히 공감해주세요.\n   → 환경을 점검하고 반복 자극을 줄이면 안정에 도움됩니다.",
            "불안": "🟤 불안감을 느끼시는 것 같아요.\n   → 어르신의 이야기를 잘 들어주시고, 따뜻한 말 한마디가 큰 위로가 됩니다.",
            "그리움": "💙 과거를 그리워하시는 마음을 표현하셨어요.\n   → 함께 옛날 이야기를 나누거나 추억 속 장소나 사람들에 대해 대화해보세요.\n   → 마음의 평안을 찾는 데 도움이 될 수 있습니다."
        }
        
        if dominant_emotion in emotion_advice:
            summary += emotion_advice[dominant_emotion]
        elif dominant_emotion in ["기쁨", "감사", "애정", "흥미"]:
            summary += "😊 긍정적인 감정을 표현하셨어요. 정말 좋네요!\n   → 이런 밝은 모습을 계속 유지하실 수 있도록 즐거운 대화와 활동을 함께 해보세요."
        elif dominant_emotion == "중립":
            summary += "💬 대부분의 대화에서 큰 감정 변화 없이 차분히 응답하셨어요.\n   → 무던해 보이지만 내면의 감정을 잘 표현하지 못하실 수도 있으니\n   → 따뜻한 말 한마디가 큰 위로가 될 수 있습니다."
        else:
            summary += "🌈 다양한 감정이 섞여 있었지만, 전반적으로 안정적인 편입니다.\n   → 지금처럼 관심과 애정을 꾸준히 표현해 주시면 좋습니다."
        
        summary += f"\n{'─'*30}\n\n"
        summary += f"📈 평가 기준\n{'─'*30}\n"
        summary += f"😊 감정 상태: 긍정적이고 안정적인 감정 표현일수록 높은 점수\n"
        summary += f"💬 답변 일관성: 질문과 관련된 적절한 답변일수록 높은 점수\n"
        summary += f"🧠 전반적 인지: 답변의 품질과 소통 능력을 종합한 점수\n"
        summary += f"{'─'*30}\n\n"
        summary += f"{'='*60}\n📋 리포트 끝 - 어르신의 건강과 행복을 위해\n{'='*60}\n"
        
        return summary
    
    def save_conversation_to_file(self, image_path=None):
        if len(self.strange_responses) == 0 and len(self.rule_based_alerts) == 0:
            self.analyze_entire_conversation()
        
        # 폴더 구조 생성
        conversation_dir = self._create_conversation_folders(image_path)
        
        # 개별 질의응답 쌍 저장 (같은 폴더 안에)
        self._save_individual_qa_pairs(conversation_dir)
        
        # 메인 대화 파일 저장: {이미지명}_conv{번호}/{이미지명}_conv{번호}.txt
        conversation_filename = conversation_dir / f"{self.conversation_id}.txt"
        with open(conversation_filename, 'w', encoding='utf-8') as f:
            f.write(f"{'='*50}\n")
            f.write(f"💬 치매 진단 대화 기록\n")
            f.write(f"{'='*50}\n")
            f.write(f"🆔 대화 ID: {self.conversation_id}\n")
            f.write(f"📊 총 대화 수: {len(self.chat_system.conversation_turns)}회\n")
            f.write(f"{'='*50}\n\n")
            
            # 대화 내용만 간단히 출력 (타임스탬프 + 대화)
            for i, turn in enumerate(self.chat_system.conversation_turns, 1):
                f.write(f"[{turn.timestamp}]\n")
                f.write(f"🤖 질문: {turn.question}\n")
                f.write(f"👤 답변: {turn.answer}\n")
                f.write(f"{'-'*30}\n\n")
        
        # analysis 폴더에 분석 리포트 저장
        analysis_dir = Path("analysis")
        analysis_dir.mkdir(exist_ok=True)
        analysis_filename = analysis_dir / f"{self.conversation_id}_analysis.txt"
        
        with open(analysis_filename, 'w', encoding='utf-8') as f:
            f.write(self.save_conversation_summary(conversation_dir))
        
        # 저장 완료 메시지
        print(f"\n✅ 파일 저장 완료!")
        print(f"📁 대화 폴더: {conversation_dir}")
        print(f"📄 대화 파일: {conversation_filename}")
        print(f"📊 분석 파일: {analysis_filename}")
        print(f"📋 QA 파일들: {len(self.chat_system.conversation_turns)}개")
        
        return str(conversation_filename), str(analysis_filename)

# 코드 통합부

In [7]:
class OptimizedDementiaSystem:
    """최적화된 치매 진단 시스템"""
    
    def __init__(self):
        self.image_analyzer = ImageAnalyzer()
        self.chat_system = ChatSystem()
        self.voice_system = VoiceSystem() if Config.SPEECH_KEY else None
        self.story_generator = StoryGenerator(self.chat_system)
    
    def analyze_and_start_conversation(self, image_path):
        """이미지 분석 및 대화 시작"""
        if not os.path.exists(image_path):
            return None
        
        # 이미지 분석
        analysis_result = self.image_analyzer.analyze_image(image_path)
        if not analysis_result:
            return None
        
        # 대화 설정
        self.chat_system.setup_conversation_context(analysis_result)
        
        # 첫 질문 생성
        initial_question = self.chat_system.generate_initial_question()
        
        return initial_question
    
    def generate_complete_analysis(self, image_path):
        """완전한 분석 생성"""
        print("\n📊 종합 분석 결과 생성 중...")
        
        # 1. 대화 기록 저장 (새로운 폴더 구조)
        conversation_file, analysis_file = self.story_generator.save_conversation_to_file(image_path)
        
        # 2. 추억 스토리 생성
        story, story_file = self.story_generator.generate_story_from_conversation(image_path)
        
        # 3. 콘솔에 요약 출력
        summary = self.story_generator.save_conversation_summary()
        print(summary)
        
        # 4. 스토리 출력
        if story:
            print(f"\n{'='*50}")
            print("📖 생성된 추억 이야기")
            print(f"{'='*50}")
            print(story)
            print(f"{'='*50}")
        
        return {
            'conversation_file': conversation_file,
            'analysis_file': analysis_file,
            'story_file': story_file,
            'story_content': story,
            'summary': summary,
            'conversation_id': self.story_generator.conversation_id
        }
    
    def _run_conversation_loop(self, image_path, is_voice=False):
        """대화 루프 실행 (음성/텍스트 공통)"""
        initial_question = self.analyze_and_start_conversation(image_path)
        if not initial_question:
            return None
        
        if is_voice and self.voice_system:
            welcome_msg = "안녕하세요. 사진을 보며 대화해요."
            print(f"🤖 {welcome_msg}")
            self.voice_system.synthesize_speech(welcome_msg)
            
            print(f"🤖 {initial_question}")
            self.voice_system.synthesize_speech(initial_question)
        else:
            print(f"🤖 {initial_question}")
        
        conversation_type = "음성" if is_voice else "텍스트"
        print(f"\n" + "="*40)
        print(f"{'🎙️' if is_voice else '💬'} {conversation_type} 대화 시작!")
        print(f"💡 {'종료라고 말하면' if is_voice else 'exit 또는 종료를 입력하면'} 끝납니다")
        print("="*40)
        
        # 대화 루프
        while True:
            if is_voice and self.voice_system:
                print("🎙️ 말씀해 주세요...")
                # 음성 녹음 시작
                self.chat_system.start_recording()
                user_input = self.voice_system.transcribe_speech()
                # 음성 녹음 중지
                audio_file = self.chat_system.stop_recording()
                
                if not user_input.strip():
                    continue
                if user_input == "종료":
                    end_msg = "대화를 마치겠습니다. 감사합니다."
                    print(f"🤖 {end_msg}")
                    self.voice_system.synthesize_speech(end_msg)
                    break
            else:
                user_input = input("\n👤 답변: ").strip()
                if user_input.lower() in ['exit', '종료', 'quit', 'q']:
                    print("대화를 종료합니다.")
                    break
            
            # AI 응답 (음성 모드일 때는 녹음된 오디오 파일 정보 전달)
            answer, should_end = self.chat_system.chat_about_image(user_input, with_audio=is_voice)
            print(f"🤖 {answer}")
            
            if is_voice and self.voice_system:
                self.voice_system.synthesize_speech(answer)
            
            if should_end:
                end_msg = "대화 시간이 종료되었습니다."
                print(f"⏰ {end_msg}")
                if is_voice and self.voice_system:
                    self.voice_system.synthesize_speech(end_msg)
                break
        
        # 종합 분석 생성
        analysis_results = self.generate_complete_analysis(image_path)
        
        if analysis_results['conversation_file']:
            print(f"📂 대화기록: {analysis_results['conversation_file']}")
            print(f"📊 분석결과: {analysis_results['analysis_file']}")
            if analysis_results['story_file']:
                print(f"📖 스토리: {analysis_results['story_file']}")
        return analysis_results
    
    def voice_conversation(self, image_path):
        """음성 대화 실행"""
        if not self.voice_system:
            return None
        return self._run_conversation_loop(image_path, is_voice=True)
    
    def text_conversation(self, image_path):
        """텍스트 대화 실행"""
        return self._run_conversation_loop(image_path, is_voice=False)

In [8]:
def interactive_voice_conversation():
    """음성 대화 실행 함수"""
    print("=== 🎤 음성 치매 진단 대화 시스템 ===")
    
    image_path = input("이미지 경로를 입력하세요: ").strip()
    
    if not image_path or not os.path.exists(image_path):
        print("❌ 올바른 이미지 경로를 입력해주세요.")
        return None
    
    try:
        system = OptimizedDementiaSystem()
        
        if not system.voice_system:
            print("❌ 음성 시스템을 초기화할 수 없습니다. Azure Speech Service 키를 확인해주세요.")
            return None
        
        return system.voice_conversation(image_path)
        
    except Exception as e:
        print(f"❌ 시스템 오류: {e}")
        return None

def interactive_conversation():
    """텍스트 대화 실행 함수"""
    print("=== 💬 텍스트 치매 진단 대화 시스템 ===")
    
    image_path = "images.jpg"
    
    if not image_path or not os.path.exists(image_path):
        print("❌ 올바른 이미지 경로를 입력해주세요.")
        return None
    
    try:
        system = OptimizedDementiaSystem()
        return system.text_conversation(image_path)
        
    except Exception as e:
        print(f"❌ 시스템 오류: {e}")
        return None

In [9]:
if __name__ == "__main__":
    # 환경 확인
    if not Config.ENDPOINT or not Config.SUBSCRIPTION_KEY:
        print("⚠️ Azure OpenAI 설정이 필요합니다:")
        print("   - gpt-endpoint")
        print("   - gpt-key")
    
    if not Config.SPEECH_KEY:
        print("⚠️ 음성 기능을 위해 Azure Speech Service 설정이 필요합니다:")
        print("   - speech-key")
    
    print("\n✅ 시스템 준비 완료!")
    print("💡 interactive_conversation() 함수를 실행해보세요!")


✅ 시스템 준비 완료!
💡 interactive_conversation() 함수를 실행해보세요!


In [10]:
interactive_voice_conversation()

=== 🎤 음성 치매 진단 대화 시스템 ===
🤖 안녕하세요. 사진을 보며 대화해요.
🤖 "어르신, 이런 축하 자리에 함께하셨던 기억이 있으신가요?"

🎙️ 음성 대화 시작!
💡 종료라고 말하면 끝납니다
🎙️ 말씀해 주세요...
👤 "아니 나는 이런 기억 없는데?"
🤖 "아, 그러셨군요. 그래도 이런 화기애애한 분위기는 참 좋아 보이지 않나요? 혹시 축하 행사라면 어떤 종류를 떠올리게 되세요?"
🎙️ 말씀해 주세요...
👤 "축하 자리라면 나는 보통 생일을 떠올르지?"
🤖 "생일이라니, 정말 따뜻하고 특별한 날이죠. 어르신께서는 기억에 남는 생일 파티가 있으셨나요?"
🎙️ 말씀해 주세요...
👤 "종료."
🤖 대화를 마치겠습니다. 감사합니다.

📊 종합 분석 결과 생성 중...
📁 저장 구조:
   메인 폴더: conversation_log/images/images_conv2/
   대화 파일: images_conv2.txt

✅ 파일 저장 완료!
📁 대화 폴더: conversation_log\images\images_conv2
📄 대화 파일: conversation_log\images\images_conv2\images_conv2.txt
📊 분석 파일: analysis\images_conv2_analysis.txt
📋 QA 파일들: 2개

📋 치매 진단 대화 분석 리포트
📅 분석 일시: 2025년 06월 08일 00:25:36
🆔 대화 ID: images_conv2

🎯 종합 평가
──────────────────────────────
😊 감정 상태:     ⭐⭐⭐⭐☆ (4/5)
💬 답변 일관성:   ⭐⭐⭐⭐⭐ (5/5)
🧠 전반적 인지:   ⭐⭐⭐⭐⭐ (5/5)
──────────────────────────────

📊 대화 개요
──────────────────────────────
💬 총 대화 횟수: 2회
😊 전반적 감정: 긍정적 (주요: 중립)
✅ 어긋난 답변: 없음
✅ 발화 패턴: 특이사항 없음
─────────────────────────────

{'conversation_file': 'conversation_log\\images\\images_conv2\\images_conv2.txt',
 'analysis_file': 'analysis\\images_conv2_analysis.txt',
 'story_file': 'story_telling\\images_story.txt',
 'story_content': '"응, 축하 자리라면 역시 생일이 가장 먼저 떠오르지. 내가 어린 시절엔 지금처럼 대단한 생일 파티는 없었어. 그래도 그때는 뭐랄까, 참 소박하고 더 따뜻했던 기억이야. 내가 열 살 되던 해, 어머니가 직접 만든 손두부를 내 생일상에 올려주시던 날이 있었지. 그 두부 위에 간장을 살짝 뿌리고, 고소한 참기름 냄새까지 더해지니 그 맛은 지금도 잊히질 않아. 다 같이 식탁에 앉아서 식구들이 웃고 떠들며 밥 먹던 그 순간이 정말 행복했거든."\n\n"그날은 비가 조금씩 내렸었는데, 우리 집 마당에 있는 대추나무 잎사귀마다 빗물이 맺혀 반짝거리던 게 아직도 눈에 선해. 아버지가 그러셨어. \'올해는 대추가 잘 열릴 거야,\' 하시면서 환히 웃으셨는데, 그 표정이 참 좋았어. 가족들과 함께 먹는 따뜻한 밥 한 끼, 그것만으로도 충분히 축하받는 기분이었어."\n\n"그리고 우리 마을에서는 생일날 친구들이 집에 와서 나무 한 그루를 심어주는 풍습도 있었는데, 내 생일에는 복숭아나무를 하나 심었어. 손으로 흙 만지면서 함께 땅을 다졌던 그 느낌이 아직도 기억나네. 그 나무가 잘 자라서 몇 년 뒤에 큰 복숭아를 주렁주렁 열었지! 그걸 아버지가 따서 한입 베어물던 날, \'우리 아들은 복숭아처럼 달콤한 인생을 살게 될 거야\' 하시던 말씀도 마음 깊이 새겨져 있어."\n\n"그래서인지 나는 생일 하면 누군가를 축하한다는 것뿐 아니라, 가족이 함께한다는 그 분위기가 떠올라. 너희도 나중에 생일을 맞이할 때, 뭘 먹고 어떻게 축하받든 가족들이 함께 있다는 사실이 얼마나 소중한지 꼭 기억했으면 좋겠구나. 요즘은 생일도